# E-Commerce Product Range Analysis

## Introduction

### Goals

The goal of this project is to analyze the product range of an online store. 

To do so we shall analyze the following: 

 - **Basket Analysis**: We will find items that are typically bought together in the same invoice. 
 
 - **Assortment**: Using keywords in the product description we will try to identify the main categories of products. 
 
 - **Returned Items**: We will analyze the distribution of the ratio of returns of items and identify the products with highest ratio. 
 
 - **Popularity**: We will identify the most popular items, and analyze distribution of popularity. 
 
 - **Seasonality**: We will plot revenues and average invoice sizes by date to see if there is any seasonal pattern. 

### Link for PDF Presentation

https://drive.google.com/file/d/1MuLi6QFhrnAz6mI1vVJV3-kY0eUbZFk0/view?usp=sharing

### Link for Tableau Dashboard

https://public.tableau.com/views/FinalProjectDashboard_16437228832330/DailyPurchases?:language=en-GB&publish=yes&:display_count=n&:origin=viz_share_link

**Description of the data:**

The dataset contains the transaction history of an online store that sells household goods.

The file `ecommerce_dataset_us.csv` contains the following columns:

`InvoiceNo` — order identifier

`StockCode` — item identifier

`Description` — item name

`Quantity`

`InvoiceDate` — order date

`UnitPrice` — price per item

`CustomerID`



### Importing Libraries

In [ ]:
import pandas as pd


import plotly.express as px

import datetime as dt 

# !pip install mlxtend

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


# import matplotlib.pyplot as plt


# !pip install pymystem3

from pymystem3 import Mystem

from collections import Counter

import numpy as np

from scipy import stats as st

### Downloading the Data

In [ ]:
try:
    df = pd.read_csv('C:/Users/tucan/OneDrive/Desktop/DATA SCIENCE/Final Project/data.csv', sep = '\t', engine = 'python')
except:
    df = pd.read_csv('/datasets/ecommerce_dataset_us.csv', sep = '\t')

### Studying the Data

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.info(memory_usage = 'deep')

In [ ]:
df.describe(include = 'all', datetime_is_numeric=True)

Here we notice that Quantity and Unit Price data have abnormal data: both negative and extremely high values. This issue will be addressed in the next section.

## Preprocessing

#### Checking for Missing Values

We see that CustomerID and Description columns have missing values. Let's see what the share of missing values is. 

In [ ]:
print('Percentage of Missing Values:')
(df.isna().sum() / len(df))*100

A quarter of all data is lacking a value in the Customer ID column. Clearly we cannot drop such an amount of data. 

We saw earlier that Customer IDs range from 12346 to 18287. If we assign a "0" to all missing customer IDs, we can identify them as "unknown customer". 

In [ ]:
# Replacing missing Customer IDs with a 0

df.CustomerID.fillna(0, inplace = True)

The Description column has a 0.27% of missing values. This is a relatively small amount of data, so we can simply delete these rows. 

In [ ]:
# Dropping rows with missing description

df.dropna(inplace = True)

In [ ]:
print('Percentage of Missing Values:')
(df.isna().sum() / len(df))*100

All missing values have been removed or replaced. 

#### Converting Data Types

We saw that some data types need to be converted. 

Customer ID column is a float, although it should not have any decimals. Let's convert it to integer type. 

In [ ]:
df['CustomerID'] = df.CustomerID.astype('int32')

Invoice date column has to be converted from object to datetime:

In [ ]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [ ]:
df.info()

By converting data types, we reduced memory usage to a fraction of the original size: from 158 MB down to 31 MB.

#### Creating columns for Date, Month and Revenue

Let's create columns for the date and month of the invoice using data from Invoice Date column.

In [ ]:
# Creating Date and Month Columns

df['date'] = df['InvoiceDate'].dt.date
df['month'] = df['InvoiceDate'].dt.month

Let's also calculate the total revenue by multiplying Unit Price and Quantity. 

In [ ]:
df['revenue'] = df['UnitPrice'] * df['Quantity']

### Filtering the Data

We saw earlier that the Quantity Column has some abnormal data. Let's take a closer look: 

In [ ]:
df.Quantity.describe()

The maximum and minimum values have the same absolute value but opposite signs, and have an unreasonably high value. Let's view these rows: 

In [ ]:
df[df.Quantity == 80995]

In [ ]:
df[df.Quantity == -80995]

The rows are almost identical, except the Invoice number for the row with the negative value has a "C" before the number, probably standing for "cancelled". The positive values could be a human errors. If that is the case, the negative value is used to cancel out the error. Negative values could also indicate returned items or discounts. Let's store all rows with negative quantities in a separate dataframe, called returns.  

In [ ]:
# Creating a DataFrame for Returned Items and Cancelled Invoices

returns = df[df.Quantity < 0]
returns.sample(5)

In [ ]:
print('Percentage of rows that have negative quantity values:')
((len(returns) / len(df))*100)

Now we can remove these negative values from our dataframe:

In [ ]:
df = df[df.Quantity > 0]

Let's see how our data looks like now: 

In [ ]:
df.Quantity.describe()

We have removed all negative values, but there still are many abnormal positive values. For our analysis we should filter out these errors. To decide what data is to be considered abnormal, let's look at the 99.5% quantile.

In [ ]:
df.Quantity.quantile(q=0.995)

99.5% of our rows have less than 160 units of a specific item on a single invoice. Let's consider any quantity beyond 160 units as an error. 

In [ ]:
df = df[df['Quantity'] <= 160]

By removing rows with more than 160 units we removed 0.5% of our data. 

In [ ]:
df.Quantity.describe()

We saw earlier that Unit Price also have some abnormal data. Let's have a closer look: 

In [ ]:
df.UnitPrice.describe()

Although we removed all negative quantity values and unusually high quantities, our unit price column still has abnormal values, including both negative and unusually high prices. Let's have a look at these values: 

In [ ]:
df[df.UnitPrice < 0]

There are only 2 rows with negative price, almost identical, that are an adjustment to a bad debt. Let's remove them.

In [ ]:
df = df[df.UnitPrice >= 0]

Let's see how many rows have a unit price equal to 0. 

In [ ]:
len(df[df.UnitPrice == 0])

In [ ]:
df[df.UnitPrice == 0].sample(5)

There are 555 rows with a Unit Price of 0. These items might have been given as a gift or a special promotion, or could be simply a human error. Anyhow, they represent only 0.001% of our data, so it is reasonable to remove them as well. 

In [ ]:
df = df[df.UnitPrice > 0]

Now let's look at abnormally high values for quantity: 

In [ ]:
df[df['UnitPrice'] > 3000]

We see that abnormally high unit prices are either Amazon or Postage Fees, or Invoices that were made manually. Since the latter lack details about the items sold, we won't use them for our analysis. Let's see how many invoices were made manually.  

In [ ]:
len(df[df['StockCode'] == 'M'])

There are only 314 such invoices, let's remove them. 

In [ ]:
df = df[df['StockCode'] != 'M']

Let's do the same for Amazon, Postage, Dotcom and bad debt adjustments. 

In [ ]:
print ('Number of rows for Amazon Fee:')
len(df[df['StockCode'] == 'AMAZONFEE'])

In [ ]:
df = df[df['StockCode'] != 'AMAZONFEE']

In [ ]:
print ('Number of rows for Postage:')
len(df[df['StockCode'] == 'POST'])

In [ ]:
df = df[df['StockCode'] != 'POST']

In [ ]:
print ('Number of rows for Dotcom Postage:')
len(df[df['StockCode'] == 'DOT'])

In [ ]:
df = df[df['StockCode'] != 'DOT']

In [ ]:
print ('Number of Bad Debt Adjustments:')
len(df[df['StockCode'] == 'B'])

In [ ]:
df = df[df['StockCode'] != 'B']

Now we have removed all these rows that are not relevant for our analysis, let's look again at our data, sorting by unit price. 

In [ ]:
df.sort_values(by='UnitPrice', ascending = False).head()

We can see here that the most expensive item is the Picnic Basket, costing 649.5, followed by the kitchen cabinet. 

Now our data is ready to be analyzed. 

## Analyzing the Data

### Basket Analysis

Let's start by finding items that are typically bought together in the same invoice.

To do so we will first group our data by Invoice and item description. 

In [ ]:
basket = (df.groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

In [ ]:
basket.head()

For our basket analysis we do not need to know the exact number of units of a specific item in a basket. Rather, we only need to know if the item is present or not in the invoice. Let's turn the quantity values into boolean: 0 if the item is not in the invoice and 1 if it is. 

In [ ]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

Now let's use the apriori algorithm to create a list of items and their support factor. We shall use a minimum support value of 0.03, meaning that we consider only items that appear in at least 3% of invoices.

In [ ]:
frequent_itemsets = apriori(basket_sets, min_support=0.03, use_colnames=True)

In [ ]:
frequent_itemsets.sample(5)

Now we will use the association rules algorithm to find the support, confidence and lift factors for all the item combinations. 

In [ ]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.sort_values('confidence', ascending = False)

By analyzing the combinations of items typically bought together we notice that there are mainly three types of items combinations: Bags, Teacups, and Alarm Clocks. 
When a customer buys teacups and saucers, he/she usually buys two (or more) different colors of this item. Same for bags and alarm clocks. 

### Item Categories

Now the next step is to try and categorize the items based on their description. 

To do so we will first see what are ten items that appear more often in our data. 

We will also use lemmatization to count the number of times each word appears in the description column. 

#### Top Ten Items

In [ ]:
df.Description.value_counts().head(10)

#### Lemmatizing

In [ ]:
product_list_full = df['Description'].unique()

m = Mystem()

string = '; '.join(product_list_full)

lemmas = m.lemmatize(string)


In [ ]:
# Counting number of times each word appears

counter = Counter(lemmas)
# print(counter)


Using the data from the top ten items and the most common words that appear in descriptions we will create a function that assigns a category to an item if it contains a specific keyword. 

In [ ]:
def category(description):   
    if 'BOX' in description:
        return 'BOXES'
    if 'DECORATION' in description or 'ORNAMENT' in description:
        return 'DECORATIONS AND ORNAMENTS'
    if 'BRACELET' in description or 'NECKLACE' in description or 'EARRINGS' in description:
        return 'JEWELRY'  
    if 'CARD' in description:
        return 'CARDS'
    if 'GAME' in description:
        return 'GAMES'
    if 'CLOCK' in description:
        return 'CLOCKS'
    if 'LIGHT' in description:
        return 'LIGHTS AND LIGHT HOLDERS'
    if 'BAG' in description:
        return 'BAGS'
    if 'CUP' in description:
        return 'CUPS'
    if 'MUG' in description:
        return 'CUPS'
    if 'TEA' in description:
        return 'CUPS'
    if 'COFFEE' in description:
        return 'CUPS'
    if  'CABINET' in description:
        return 'CABINET'
    if 'CHALK' in description:
        return 'CHALKBOARDS'
    if 'CUTLERY' in description:
        return 'CUTLERY'
    if 'CAKE' in description:
        return 'CAKE STANDS, CASES AND TINS'
    if 'JAM' in description:
        return 'JAM MAKING SETS'
    if 'BUNTING' in description:
        return 'BUNTING'
    if 'FRAME' in description:
        return 'FRAMES'
    if 'BOWL' in description:
        return 'BOWLS'
    if 'SHOPPER' in description:
        return 'SHOPPERS'  
    if 'HANGER' in description:
        return 'HANGERS'  
    if 'LANTERN' in description:
        return 'LIGHTS AND LIGHT HOLDERS'
    if 'BOTTLE' in description:
        return 'BOTTLES'
    if 'DOLL' in description:
        return 'TOYS AND DOLLS' 
    if 'TOY' in description:
        return 'TOYS AND DOLLS'
    if 'DOORMAT' in description:
        return 'DOORMATS'
    if 'CANDLE' in description:
        return 'CANDLES AND CANDLE HOLDERS'
    if 'PEN' in description or 'PENCIL' in description or 'STATIONERY' in description:
        return 'STATIONERY'
    else:
        return 'OTHER'

Now let's apply the function to our data. 

In [ ]:
df['Category'] = df['Description'].apply(category)

Now we can calculate the percentage of items belonging to each category:

In [ ]:
((df['Category'].value_counts()/len(df))*100).round(2)

Although we have identified the 20 most common categories of items, only 54% of items have been categorized. 
The most common category is bags, which accounts for almost 10% of all items, followed by cups, boxes and lights, which are approximately 5% each. 

Let's apply the same function to the returns dataframe. First, though, we need to filter the returns dataframe like we did for the main dataframe. 

In [ ]:
returns.UnitPrice.describe()

In [ ]:
returns[returns['UnitPrice'] > 649.5].sample(5)

In [ ]:
len(returns[returns['UnitPrice'] > 649.5])

We saw earlier that the most expensive item costs 649.5. Anything beyond that value is either a postage fee or a manual invoice. 

These rows correspond to about 0.1 % of our data. Let's filter them:

In [ ]:
returns = returns[returns['UnitPrice'] <= 649.5]

Let's have a look at the data for Quantity:

In [ ]:
returns.Quantity.describe()

In [ ]:
len(returns[returns['Quantity'] < -160])

In our main dataframe we removed all invoices for more than 160 units of an item. In returned items data about 2% of our rows have quantities lower than -160. Let's remove these rows.

In [ ]:
returns = returns[returns['Quantity'] > -160]

Now we can apply the categorizing function to returned items data. 

In [ ]:
returns['Category'] = returns['Description'].apply(category)

In [ ]:
((returns['Category'].value_counts()/len(returns))*100).round(2)

Bags are almost 10% of our main data but only 7.39% of returned items. Cups, on the contrary, were only 5% of items in our main data but 7.65% for returned items. Also cake-related items, boxes and lights had a higher ratio of returns compared to their share in the main data.  

Our next step is to analyze the seasonality of sales. 

### Analyzing Seasonality

Let's have a look at daily revenues and monthly revenues.

In [ ]:
fig = px.histogram(df, x = 'date', y ='revenue', title = 'Daily Revenues')


In [ ]:
fig.show()

By looking at the daily revenues we notice the following: 

- There are two gaps with zero sales durning Christmas and Easter holidays, the former starting from the 23rd of December to the 2nd of January, the latter from the 20th to the 24th of April. 

- From a weekly point of view, there are no sales on Sundays. 

Our data covers a period of slightly more than one year (from the 29th of November 2018 until December 7th 2019). 

Let's view our revenue histogram with 13 bins to see if there is any seasonal pattern.

In [ ]:
fig = px.histogram(df, x = 'date', y ='revenue', title = 'Monthly Revenue', nbins = 13)
fig.show()

Ignoring November 2018 and December 2019 (since data for these months is incomplete), we can clearly see that November is the month with the highest revenue (84.000), followed by October with 61.000. All other months have a total revenue ranging from 25.000 to 46.000. 

### Item Popularity Analysis

First let's look at the general distribution of popularity of each item: 

In [ ]:
grouped_by_item = df.pivot_table(index = 'Description', 
                                 values = ['InvoiceNo', 'UnitPrice'], 
                                 aggfunc = {'InvoiceNo':'count',
                                            'UnitPrice' : 'mean'}).round(2)

grouped_by_item.rename(columns = {'InvoiceNo' : 'Purchases'}, 
                       inplace = True)

grouped_by_item = grouped_by_item.sort_values('Purchases', ascending = False).reset_index()

grouped_by_item.Purchases.describe()

An item is purchased on average 130 times a year. 

Let's have a look at the ten items that appear most often in invoices

In [ ]:
df.Description.value_counts().head(10)

Now let's look at the items that sold the most units.

In [ ]:
quantity = df.pivot_table(index = ['StockCode','Description'],
                       values = 'Quantity',
                       aggfunc = 'sum')

quantity.sort_values(by = 'Quantity', ascending = False).head(10)

By comparing these two lists we can see some items that do not appear in the top ten in the list of items per invoice, but appear in the top ten when considering number of units sold, like the WORLD WAR 2 GLIDERS and POPCORN HOLDERS. Let's have a look at the average number of units of each item per invoice. 

In [ ]:
avg_quantity = df.pivot_table(index = ['StockCode','Description'],
                       values = 'Quantity',
                       aggfunc = 'mean').astype('int')
avg_quantity.rename(columns={'Quantity' : 'Average Number of Units'}, inplace=True)
avg_quantity.sort_values(by = 'Average Number of Units', ascending = False).head(10)

In [ ]:
avg_quantity.describe()

The item than on average sells the highest number of units is Mini Highlighter, with an average of 120 units per invoice and we see several other stationary items in the top ten of the list. 

### Price Range Analysis

Now let's analyze the distribution of price per unit. 

In [ ]:
df.UnitPrice.describe()

We know that the most expensive item costs 649.5, which is about 200 times the average cost per unit. 

Let's calculate the 99% quantile: 

In [ ]:
df.UnitPrice.quantile(0.99)

Only 1% of items are more expensive than 16.63. 

Let's see which are the top ten most expensive items: 

In [ ]:
price = df.pivot_table(index = ['StockCode','Description'],
                       values = 'UnitPrice').round(2)
# avg_quantity.rename(columns={'UnitPrice' : 'Price per Unit'}, inplace=True)
price.sort_values(by = 'UnitPrice', ascending = False).head(10)

The second most expensive item, the Sideboard, is only a quarter of the price of the most expensive item, the Picnic basket. 

Let's plot a histogram to analyze the distribution of prices of items cheaper than 17, which as we saw are 99% of our products. 

In [ ]:
fig = px.histogram(price[price['UnitPrice'] <= 17], x = 'UnitPrice',  title = 'Distribution of Item Prices', nbins = 16)
fig.show()

Approximately half of all items are within the price range of 1 and 3. A minority of items cost beyond 5.  

### Invoice Variety

Let's explore the variety of invoices by total revenue and number of items. 

In [ ]:
invoices = df.pivot_table(index = ['InvoiceNo', 'date'],
                                 values = ['revenue', 'StockCode', 'Quantity', 'UnitPrice'],
                                 aggfunc = {'revenue': 'sum',
                                          'StockCode': 'count',
                                          'Quantity' : 'sum',
                                           'UnitPrice' : 'mean'})

invoices.rename(columns= {'StockCode': 'Items',
                        'Quantity' : 'Units',
                         'UnitPrice' : 'Average Product Price'}, 
                        inplace = True)

invoices.sort_values('revenue', ascending = False, inplace = True)

invoices.reset_index(inplace = True)

In [ ]:
invoices

In [ ]:
invoices.revenue.describe()

Median invoice size is 300 and average size is 450. A few very large outliers are pulling the average higher than the median. As we can see the biggest invoice is almost 40.000. Let's compute the 95% quantile and plot a histogram for invoice sizes of 95% of invoices. 

In [ ]:
invoices.revenue.quantile(.95).round(2)

In [ ]:
fig = px.histogram(invoices[invoices['revenue'] < 1321], 
                   x = 'revenue',  
                   title = 'Distribution of Invoice Revenue', 
                   nbins = 20)
fig.show()

The revenue range that has the highest number of invoices is between 250 and 350. 

Now let's analyze the seasonality of invoices by revenue and number of items.

In [ ]:
invoice_by_date = invoices.pivot_table(index = 'date',
                                                values = ['revenue', 'Items', 'Units'],
                                                aggfunc = 'mean').round(2).reset_index()


In [ ]:
fig = px.histogram(invoice_by_date, 
                   x = 'date',
                   y = 'revenue',
                   title = 'Seasonality of Invoice Size by Revenue', 
                   nbins = 50)
fig.show()

As we saw earlier there are short winter and Easter breaks were there are 0 sales, so that explains the two gaps with lower revenue. Also we know that October and November are the months with highest overall revenue, and we can see that the average revenue per invoice in that period is only slightly higher than the rest of the year, suggesting that October and November mainly have a larger number of customer, rather than the same number of customers spending more money compared to other times of the year. 

Although January is not one of the most profitable months from an overall revenue point of view, we notice the average revenue per invoice seems to be higher than average specifically in the period within the 7th and 19th of January. 

Now let's analyze the seasonality of invoice size by number of items bought. 

In [ ]:
invoices.Items.describe()

In [ ]:
fig = px.histogram(invoice_by_date, 
                   x = 'date',
                   y = 'Items',
                   title = 'Seasonality of Invoice Size by Number of Items', 
                   nbins = 40)
fig.show()

By plotting the sum of average number of items per day we notice the peak in the period from the 6th to the 19th of January is even more obvious. We shall test later if this difference is statistically significant. 

### Returned Items

Let's create a table for the number of returns per item: 

In [ ]:
returns_grouped_by_item = returns.pivot_table(index = 'Description', 
                                              values = ['InvoiceNo'], 
                                              aggfunc = 'count')

returns_grouped_by_item.rename(columns = {'InvoiceNo' : 'Returns'}, inplace = True)

returns_grouped_by_item = returns_grouped_by_item.sort_values('Returns', ascending = False).reset_index()


Let's find the return ratio of each product, by merging the table of returned items with the list of all products. 

In [ ]:
return_ratio = grouped_by_item.merge(returns_grouped_by_item, on = 'Description', how = 'left')

return_ratio['return_ratio'] = ((return_ratio["Returns"] / return_ratio["Purchases"])*100).round(1)

return_ratio.fillna(0, inplace = True)

Let's start by studying the general return ratio:

In [ ]:
return_ratio.return_ratio.describe()

We see that there are some abnormally high ratios. Logically an item cannot be returned more than 100% of the times it was purchased. Let's view return ratios that are beyond 100%. 

In [ ]:
return_ratio.sort_values(by=['return_ratio'], ascending = False).head(10)

Nine rows in returned items have a rate higher than 100%, but only six of these are actually products (the others are Manual, Samples and Bank Charges). These products all sold a very small number of units, and the returned units have probably been purchased before the analyzed period. So let's filter these rows and view the statistics after filtering. 

In [ ]:
return_ratio = return_ratio[return_ratio['return_ratio'] <= 100]

In [ ]:
return_ratio['return_ratio'].describe()

We can see that the majority of items were not returned even once, and the average return ratio is 1.8%. 

Let's view the items that were returned the most: 

In [ ]:
return_ratio.sort_values(by=['return_ratio'], ascending = False).head(10)

We see that products with a return ratio of 100% or 50% typically were purchased only once or twice. 
Let's find the 5% quantile for number of purchases, and view items that were bought more than that value: 

In [ ]:
return_ratio['Purchases'].quantile(0.05)

95% of items were purchased more than twice. Let's view the items with highest return ratio that sold more than twice: 

In [ ]:
return_ratio[return_ratio['Purchases'] > 2].sort_values(by=['return_ratio', 'Purchases'], ascending = False).head(10)

Now let's look at the return ratio for the ten most popular items: 

In [ ]:
return_ratio.sort_values(by=['Purchases', 'return_ratio'], ascending = False).head(10)

Of the ten most popular items, most have a very low return ratio, except two: REGENCY CAKESTAND 3 TIER and SET OF 3 CAKE TINS PANTRY DESIGN, both of them belonging to the "cake" category. 

Now let's look at the return ratio for the ten most expensive items: 

In [ ]:
return_ratio.sort_values(by=['UnitPrice'], ascending = False).head(10)

Of the most expensive items, we notice two vintage kitchen cabinets, red and blue, have a very high return ratio. The vintage post office cabinet, however, has 0 returns. 

### Testing Hypotheses

We saw earlier that there was a peak in the sum of the average number of items during the two week period from the 6th of January to the 19th. Now we will test if the average number of items per invoice in this period is significantly bigger than the average for the rest of the year.

First let's create a dataframe for the period we want to test: 

In [ ]:
test_period = df[df['InvoiceDate'] > '2019-01-06']
test_period = test_period[test_period['InvoiceDate'] <= '2019-01-20']

Now we can calculate the average number of items per invoice and their variance, for both test and control period.  

In [ ]:
test_period_items = test_period.groupby('InvoiceNo')['StockCode'].count()

print('Average Number of Items per invoice in test period: {0:.1f}'.format(test_period_items.mean())) 

print('Variance of number of items in test period: {0:.1f}'.format(np.var(test_period_items)))


In [ ]:
print('Average Number of Items per invoice in whole data: {0:.1f}'.format(invoices.Items.mean()))

print('Variance of number of items in whole data: {0:.1f}'.format(np.var(invoices.Items)))


Now we can test our hypothesis:

 - HO: There is no difference in number of items per invoice in the period from the 6th and the 19th of January compared to the average of the whole year. 

 - H1: In the period between the 6th and 19th of January, invoices included a higher than average number of items. 

In [ ]:
alpha = 0.05

results = st.ttest_ind(test_period_items, invoices.Items, equal_var = False) 

print('p-value:', results.pvalue) 

if (results.pvalue < alpha): 
    print("We reject the null hypothesis")
else:
    print("We can't reject the null hypothesis")

In the period we tested, invoices had on average 39 items. This value is significantly higher than the rest of the year, with an average of 27. 

However, we saw earlier that the sum of average revenues in January was actually one of the lowest of the year. This suggests that although the part of January we tested had more items in each invoice, the same period might not have a significantly higher average revenue per invoice. 

Let us compare the revenue of our test period with the average revenue per invoice for the whole year. 

In [ ]:
test_period_revenue = test_period.groupby('InvoiceNo')['revenue'].sum()

print('Average Revenue per invoice in test period: {0:.1f}'.format(test_period_revenue.mean())) 
print('Variance of revenue in test period: {0:.1f}'.format(np.var(test_period_revenue)))

In [ ]:
print('Average Revenue per invoice in whole data: {0:.1f}'.format(invoices.revenue.mean())) 

print('Variance of revenue in whole data: {0:.1f}'.format(np.var(invoices.revenue)))


The average invoice in the test period is larger than average, but is this difference statistically significant? Let's test the following hypothesis: 

 - HO: The average revenue per invoice in the period between the 6th and the 19th of January is the same as the average invoice of the whole year
 - H1: In the period  between the 6th and the 19th of January the average revenue per invoice is higher than average

In [ ]:
alpha = 0.05

results = st.ttest_ind(test_period_revenue, invoices.revenue, equal_var = False) 

print('p-value:', results.pvalue) 

if (results.pvalue < alpha): 
    print("We reject the null hypothesis")
else:
    print("We can't reject the null hypothesis")

The difference is not statistically significant, meaning that the period in January we are testing is not characterized buy larger invoices, but just invoices with a higher number of items. The P-value, however, is just slightly higher than our level of statistical significance, 0.05. 

#### Conclusion

The two week period we tested is characterized by invoices with a larger variety, but not significantly higher revenues. 

## Final Conclusions


#### Preparing the Data

After studying and preprocessing our data, we removed from our data all rows that do not correspond to a specific products.  Some of these rows were charges for different kinds of fees like posting or Amazon fees. Some rows were manual invoices, that do not have a list of products, but just the total sum of the purchase. 
Then we found rows with negative prices and identified them as returned items, and stored them in a separate table. 

#### Categorization

We then tried to identify the 20 most common categories of products using the most common keywords present in the item description. We managed to categorize slightly more than half of all items. The most common categories are Bags, Cups and Boxes.

#### Price Range

Most of the products are very cheap, within the range of 1 and 3. Only 1% of items cost more than 16.63. The most expensive product costs 649.50: the PICNIC BASKET WICKER 60 PIECES. 	

#### Popularity

We saw that the average product appears 130 times per year in invoices, but the median value is only 63, indicating that some very popular items are being sold much more than average. The most popular item, the WHITE HANGING HEART T-LIGHT HOLDER appears 2304 times in invoices. We then checked the average number of units bought for the same item in an invoice, and found that, on average, 6.7 units of an item are sold per invoice.    

#### Returned Items 

We compared the ranking of the most common categories with the shares of categories amongst returned items and found that bags have a relatively lower chance of being returned, while cups, boxes and cake-related items tend to have a slightly higher chance of being returned.
We analyzed the ratio of return of all items, and found that the average return ratio is 1.8 %, and the majority of products were not returned even once. We selected items that sold at least 10 times during the analyzed year and viewed which of these products had the highest return ratios. The item with the highest ratio was DANISH ROSE BEDSIDE CABINET, with a 50% return ratio. Of the top ten products, three of them belong to the cabinet category. We then checked the ratio amongst the most popular items, and found that most of these had very low ratios. Of these products, the one with the highest ratio was REGENCY CAKESTAND 3 TIER with 9% being returned. When looking at the ten most expensive items, we notice again two items belonging to the cabinet category with very high return ratios: the VINTAGE KITCHEN CABINETS in blue and red. 

#### Basket Analysis

We performed a Basket Analysis and found the items that have are typically being bought together in the same purchase. We noticed that the items that are most typically being bought together are the same product in different colours. These products are: 

 - REGENCY TEACUPS AND SAUCERS 
 - ALARM CLOCK BAKELIKE
 - JUMBO BAG RETROSPOT

It could be a good idea to offer customers a special deal on an assortment of different colors for these products.

#### Seasonality

We analysed the overall seasonality of revenues and found that the most profitable months are November and October. 
We also noticed that there are zero sales on Sundays. The same is true for winter and Easter holidays.
We then anayzed the seasonality of the average size of invoices by average number of items and revenue, and noticed a peak on the two week period from the 6th to the 19th of January. We checked if this increase in average number of items and revenue is significantly higher than the yearly average, and we found that the number of items is significantly higher, while average revenues were not. There could be two possible explanations for this: 

 - 1) During this period customers tend to buy cheaper items with a larger variety
 
 - 2) During this period customers tend to buy more items with the same average price but less units of each item
 
It could be interesting to look deeper into this aspect. This could be a subject for deeper analysis. 

#### Sources: 

For the Basket Analysis I was helped by the following articles: 

https://www.analyticsvidhya.com/blog/2021/10/a-comprehensive-guide-on-market-basket-analysis/

https://pbpython.com/market-basket-analysis.html

https://analyticsindiamag.com/beginners-guide-to-understanding-apriori-algorithm-with-implementation-in-python/

https://medium.com/@jihargifari/how-to-perform-market-basket-analysis-in-python-bd00b745b106